In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from skimage.measure import label, regionprops
import cartopy.crs as ccrs
import warnings

# Filter out FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Load data
# Replace with seaice file path
df = xr.open_dataset('sea_ice_20022024.nc').squeeze()


# Load pixel area data
pixel_areas = xr.open_dataset(
    'NSIDC0771_CellArea_PS_S6.25km_v1.0.nc')['cell_area']

# Load lat-lon data for each x-y coordinate
# Replace with your lat-lon file path
latlon_data = xr.open_dataset('NSIDC0771_LatLon_PS_S6.25km_v1.0.nc')
latitudes = latlon_data['latitude'].values
longitudes = latlon_data['longitude'].values

# Mask data
mask = df.z < 30  # Polynya Indentification Threshold

# Label connected components
mask_labeled = np.vectorize(label, signature='(n,m)->(n,m)')(mask)

# Initialize DataFrame
properties_list = []


def calculate_region_properties(labeled_data, timestamp):
    properties = []
    for region in regionprops(labeled_data, intensity_image=pixel_areas):
        if region.area > 1:  # Ensure valid regions
            total_area = region.intensity_image.sum().item()

            # Get centroid coordinates
            y, x = region.centroid
            lat = latitudes[int(y), int(x)]
            lon = longitudes[int(y), int(x)]

            properties.append({
                'label': region.label,
                'lat': lat,
                'lon': lon,
                'total_area': total_area,
                'Timestamp': pd.to_datetime(timestamp)
            })
    return properties


# Loop through each timestamp
for idx, timestamp in enumerate(df.time):
    properties = calculate_region_properties(
        mask_labeled[idx], timestamp.values)
    properties_list.extend(properties)

# Convert to DataFrame
properties_df = pd.DataFrame(properties_list)

# Print the result
print(properties_df)

properties_df.to_csv('Polynya_Area_30.csv')